In [10]:
#INI KOMPAS
import os
import csv
import requests as req
from bs4 import BeautifulSoup as bs
from datetime import datetime
from fake_useragent import UserAgent

ua = UserAgent()
hades = {'User-Agent': ua.random}

# Cek apakah User-Agent mengandung 'Windows'
while 'Windows' not in hades['User-Agent']:
    hades = {'User-Agent': ua.random}

def scrape_kompas(hal):
    global hades
    a = 0
    file_path = 'kompas1.csv'
    
    # Cek apakah file sudah ada
    file_exists = os.path.isfile(file_path)
    
    # Rentang tahun yang diinginkan (hanya tahun 2025)
    target_year = 2025

    for page in range(1, hal + 1):  # +1 agar loop bisa mengakses halaman yang sesuai
        url = f'https://indeks.kompas.com/?site=news&page={page}'
        ge = req.get(url, hades).text
        sop = bs(ge, 'html.parser')
        li = sop.find('div', class_='articleList -list')
        
        if li:
            lin = li.find_all('div', class_='articleItem')

            with open(file_path, 'a', newline='', encoding='utf-8') as file:
                wr = csv.writer(file, delimiter=',')
                
                # Jika file belum ada, tulis header terlebih dahulu
                if not file_exists:
                    wr.writerow(["headline", "date", "link", "content"])
                    file_exists = True  # Hindari penulisan header ulang

                for x in lin:
                    link = x.find('a')['href']
                    headline = x.find('h2', class_='articleTitle').text.strip()
                    date_div = x.find('div', class_='articlePost-date').text.strip()  # Contoh: "18/02/2025"
                    
                    # Parsing tanggal
                    try:
                        # Mengonversi string tanggal menjadi objek datetime dengan format dd/mm/yyyy
                        date = datetime.strptime(date_div, "%d/%m/%Y")
                    except Exception as e:
                        print(f"Error parsing date: {e}")
                        continue
                    
                    # Filter hanya untuk artikel tahun 2025
                    if date.year != target_year:
                        continue  # Lewati artikel yang tidak sesuai dengan tahun 2025
                    
                    ge_ = req.get(link, hades).text
                    sop_ = bs(ge_, 'html.parser')
                    content = sop_.find_all('div', class_='read__content')

                    for x in content:
                        x = x.find_all('p')
                        y = [y.text for y in x]
                        content_ = ' '.join(y).replace('\n', '').replace('ADVERTISEMENT', '').replace('SCROLL TO CONTINUE WITH CONTENT', '')

                        print(f'done[{a}] > {headline[:10]}')
                        a += 1
                        wr.writerow([headline, date.strftime("%d/%m/%Y"), link, content_])

import lxml
scrape_kompas(666)


done[0] > Menhut Raj
done[1] > Usai Porak
done[2] > Polisi Bon
done[3] > Kenaikan P
done[4] > KPK Versus
done[5] > Hotman Par
done[6] > Jaksa Agun
done[7] > Kala Para 
done[8] > Misteri Ja
done[9] > Razman Nas
done[10] > Razman Nas
done[11] > Kemendes S


KeyboardInterrupt: 

In [ ]:
# INI KOMPAS baru
import os
import csv
import requests as req
from bs4 import BeautifulSoup as bs
from datetime import datetime, timedelta
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import csv
import os
import requests as req
from bs4 import BeautifulSoup as bs
import re
from fake_useragent import UserAgent
from webdriver_manager.chrome import ChromeDriverManager

# Konfigurasi Chrome WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless")  
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

def scrape_kompas():
    file_path = 'kompas_scrape1.csv'
    file_exists = os.path.isfile(file_path)
    
    # Menulis header jika file belum ada
    with open(file_path, 'a', newline='', encoding='utf-8') as file:
        wr = csv.writer(file, delimiter=',')
        if not file_exists:
            wr.writerow(["headline", "date", "link", "content"])
    
    start_date = datetime(2025, 2, 20)
    end_date = datetime(2025, 2, 28)
    delta = timedelta(days=1)
    
    ua = UserAgent()
    hades = {'User-Agent': ua.random}
    keywords = ["MBG", "Makanan Bergizi Gratis", "makan bergizi gratis"]

    
    # Loop untuk setiap tanggal dalam rentang waktu yang diinginkan
    while start_date <= end_date:
        formatted_date = start_date.strftime('%Y-%m-%d')
        print(f"Scraping date: {formatted_date}")
        
        for page in range(1, 21):  # Maksimal 20 halaman per tanggal
            url = f'https://indeks.kompas.com/?site=news&date={formatted_date}&page={page}'
            driver.get(url)
            time.sleep(3)
            soup = bs(driver.page_source, 'html.parser')
            li = soup.find('div', class_='articleList -list')
            articles = li.find_all('div', class_='articleItem')
            
            if not articles:
                print(f"Tidak ada artikel di halaman {page}, lanjut ke tanggal berikutnya.")
                break
            
            with open(file_path, 'a', newline='', encoding='utf-8') as file:
                wr = csv.writer(file, delimiter=',')
                
                for article in articles:
                    headline_elem = article.find('h2', class_='articleTitle')
                    link_elem = article.find('a')
                    date_elem = article.find('div', class_='articlePost-date')
                    
                    if not (headline_elem and link_elem and date_elem):
                        continue
                    
                    headline = headline_elem.text.strip()
                    link = link_elem['href']
                    date_text = date_elem.text.strip()
                    if not any(keyword.lower() in headline.lower() for keyword in keywords):
                        continue
                    
                    # Mengambil konten artikel
                    article_resp = req.get(link, headers=hades)
                    article_soup = bs(article_resp.text, 'html.parser')
                    content_div = article_soup.find('div', class_='read__content')
                    
                    content = ''
                    if content_div:
                        paragraphs = content_div.find_all('p')
                        content = ' '.join(p.text.strip() for p in paragraphs if p.text.strip())
                    
                    print(f"Scraped: {headline[:50]}... | Date: {date_text}")
                    wr.writerow([headline, date_text, link, content])
        
        # Pindah ke tanggal berikutnya
        start_date += delta

scrape_kompas()


Scraping date: 2025-02-20
Scraped: Tak Ada Kodim, Pemkab Pegunungan Bintang Belum Bis... | Date: 20/02/2025
Scraping date: 2025-02-21
Scraped: Makan Bergizi Gratis: Mengurai Kompleksitas di Bal... | Date: 21/02/2025
Scraped: Tolak Makan Bergizi Gratis, Apakah Haram?... | Date: 21/02/2025
Tidak ada artikel di halaman 15, lanjut ke tanggal berikutnya.
Scraping date: 2025-02-22
Tidak ada artikel di halaman 7, lanjut ke tanggal berikutnya.
Scraping date: 2025-02-23
Tidak ada artikel di halaman 6, lanjut ke tanggal berikutnya.
Scraping date: 2025-02-24
Scraped: Kepala Badan Gizi: Murid Yang Tak Puasa Bisa Santa... | Date: 24/02/2025
Scraped: BGN Bakal Beri Materi Peran Pemda dalam Program MB... | Date: 24/02/2025
Scraped: Temui Prabowo, Kepala Badan Gizi Laporkan MBG Suda... | Date: 24/02/2025
Scraped: Menu MBG Selama Ramadhan Dibuat Tak Mudah Basi: Te... | Date: 24/02/2025
Scraped: Makan Bergizi Gratis di 8 Sekolah Cilincing Batal ... | Date: 24/02/2025
Scraped: Makan Bergizi Gratis Tetap 

In [38]:
import pandas as pd

dt = pd.read_csv('kompas_scrape.csv')
dt = dt.dropna()

dt['content'] = dt['content'].str.partition('-')[2].str.strip()
# Filter headline yang mengandung "MBG" atau "Makanan Bergizi Gratis"
dt_filtered = dt[dt['headline'].str.contains(r'\b(?:MBG|BGN|Makanan\s*Bergizi\s*Gratis|Makan\s*Bergizi\s*Gratis|Makan\s*Gratis)\b', case=False, na=False)]
dt_filtered = pd.DataFrame(dt_filtered)
dt_filtered.to_csv('kompas_scrapef2.csv', index=False, encoding='utf-8')

In [39]:
dt1 = pd.read_csv('kompas_scrapef2.csv')
dt1 = dt1.dropna()
dt2 = pd.read_csv('kompas_scrapef1.csv')
dt2 = dt2.dropna()

df_combined = pd.concat([dt1, dt2], ignore_index=True)
df_combined.to_csv('kompasfixbgt1.csv', index=False, encoding='utf-8')

In [ ]:
# CEK KOMPAS
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import csv
import os
import requests as req
from bs4 import BeautifulSoup as bs
import re
from fake_useragent import UserAgent
from webdriver_manager.chrome import ChromeDriverManager

# Konfigurasi Chrome WebDriver
chrome_options = Options()
chrome_options.add_argument("--headless")  
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
ua = UserAgent()
hades = {'User-Agent': ua.random}

url = f'https://indeks.kompas.com/?site=news&date=2025-01-07&page=2'
driver.get(url)
time.sleep(3)
soup = bs(driver.page_source, 'html.parser')
li = soup.find('div', class_='articleList -list')
#print(li)
articles = li.find_all('div', class_='articleItem')
print(articles)

[<div class="articleItem">
<a class="article-link" href="https://megapolitan.kompas.com/read/2025/01/07/20333561/hasto-tidak-ada-di-rumah-bekasi-saat-kpk-melakukan-penggeledahan">
<div class="articleItem-wrap">
<div class="articleItem-img">
<img alt="Hasto Tidak Ada di Rumah Bekasi Saat KPK Melakukan Penggeledahan" src="https://asset.kompas.com/crops/_Pefqa2nQMWdbr3FhJ3T1r7Byic=/0x0:0x0/230x152/data/photo/2025/01/07/677cf224d1dd5.jpeg"/>
</div>
<div class="articleItem-box">
<h2 class="articleTitle">Hasto Tidak Ada di Rumah Bekasi Saat KPK Melakukan Penggeledahan</h2>
<div class="articlePost">
<ul>
<li>
<div class="articlePost-subtitle">Megapolitan</div>
</li>
<li>
<div class="articlePost-date">07/01/2025</div>
</li>
</ul>
</div>
</div>
</div>
</a>
</div>, <div class="articleItem">
<a class="article-link" href="https://megapolitan.kompas.com/read/2025/01/07/20270821/hasto-kristiyanto-kembali-diperiksa-kpk-pada-13-januari-2025">
<div class="articleItem-wrap">
<div class="articleItem-img"

In [147]:
import pandas as pd

dt = pd.read_csv('kompas.csv')
dt = dt.dropna()

dt['content'] = dt['content'].str.partition('-')[2].str.strip()
# Filter headline yang mengandung "MBG" atau "Makanan Bergizi Gratis"
dt_filtered = dt[dt['headline'].str.contains(r'\b(MBG|Makanan Bergizi Gratis)|Makan Bergizi Gratis\b', case=False, na=False)]
dt_filtered = pd.DataFrame(dt_filtered)
dt_filtered.to_csv('kompasfixx.csv', index=False, encoding='utf-8')


C:\Users\raziq\AppData\Local\Temp\ipykernel_27460\3425639866.py:8: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  dt_filtered = dt[dt['headline'].str.contains(r'\b(MBG|Makanan Bergizi Gratis)|Makan Bergizi Gratis\b', case=False, na=False)]


In [63]:
#INI TEMPO FIX

import os
import csv
import time
import re
import requests as req
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup as bs
from fake_useragent import UserAgent

def scrape_tempo(pages):
    # Konfigurasi WebDriver
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")

    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    
    # Setup User-Agent Palsu
    ua = UserAgent()
    headers = {'User-Agent': ua.random}
    
    file_path = 'tempo_fix.csv'
    file_exists = os.path.isfile(file_path)

    with open(file_path, 'a', newline='', encoding='utf-8') as file:
        wr = csv.writer(file, delimiter=',')
        if not file_exists:
            wr.writerow(["headline", "date", "link", "content"])

        base_url = "https://www.tempo.co/search?q=%23mbg&page=1"
        driver.get(base_url)

        for page in range(1, pages + 1):
            print(f"Scraping page {page}...")

            time.sleep(3)
            soup = bs(driver.page_source, "html.parser")
            articles = soup.select("div.flex.flex-col.divide-y > div")
            
            if not articles:
                print(f"Tidak ada artikel di halaman {page}")
                break
            
            for article in articles:
                headline_elem = article.select_one("p.text-neutral-1200 a")
                if not headline_elem:
                    continue  

                link = "https://www.tempo.co" + headline_elem["href"]
                headline = headline_elem.text.strip()

                # Mengambil konten berita menggunakan requests
                response = req.get(link, headers=headers)
                news_soup = bs(response.text, "html.parser")

                # Ambil tanggal publikasi
                date_div = news_soup.find("p", class_="text-neutral-900 text-sm")
                date_match = re.search(r"([\d]{1,2} \w+ \d{4})", date_div.text if date_div else "")
                date = date_match.group(1) if date_match else "Unknown"

                # Ambil isi berita
                content_div = news_soup.find('div', class_='max-lg:container xl')
                if content_div:
                    paragraphs = content_div.find_all("p")
                    content = ' '.join([p.text.strip() for p in paragraphs])
                    content = re.sub(r'\s+', ' ', content).strip()
                else:
                    content = ""

                print(f"Scraped: {headline[:50]}... | Date: {date}")
                wr.writerow([headline, date, link, content])

            # Klik tombol Next jika ada
            try:
                next_button = driver.find_element("xpath", "//button[@aria-label='Next Page']")
                driver.execute_script("arguments[0].click();", next_button)
                time.sleep(3)
            except Exception:
                print("Tidak ada tombol Next Page, berhenti.")
                break

    driver.quit()

# Scrape 5 halaman pertama
scrape_tempo(50)


Scraping page 1...
Scraped: Menko Pangan Sebut MBG Serap Anggaran Hingga Rp 2 ... | Date: 3 Maret 2025
Scraped: Badan Gizi Nasional Akan Biayai Pengobatan Siswa y... | Date: 28 Februari 2025
Scraped: Program MBG di Tulang Bawang Dilakukan Secara Bert... | Date: 27 Februari 2025
Scraped: Temuan Ayam Mentah di Makan Bergizi Gratis di SD S... | Date: 27 Februari 2025
Scraped: Keliru: Dua Foto Pelajar SD di Papua Meninggal kar... | Date: 24 Februari 2025
Scraped: Murid SDN Haurpancuh Bandung Santap Makan Bergizi ... | Date: 24 Februari 2025
Scraped: Evaluasi Kasus Keracunan Makan Bergizi Gratis, Ter... | Date: 21 Februari 2025
Scraped: Dosen Fisipol UGM: Indonesia Gelap Respons Masyara... | Date: 21 Februari 2025
Scraped: Pelajar di Wamena Papua Pegunungan: Tolak Makan Be... | Date: 19 Februari 2025
Scraped: Rangkaian Persoalan Makan Bergizi Gratis di Berbag... | Date: 19 Februari 2025
Scraping page 2...
Scraped: Aksi Indonesia Gelap: BEM Universitas Trisakti Sua... | Date: 19 Februari 202

In [28]:
import pandas as pd

# Baca file hasil scraping
file_path = "tempo_fix.csv"
df = pd.read_csv(file_path)
df = df.dropna()

# Hapus duplikat berdasarkan headline
df = df.drop_duplicates(subset=["headline"], keep="first")

# Filter hanya yang judulnya mengandung 'MBG' atau 'Makan Bergizi Gratis'
df_filtered = df[df['headline'].str.contains(r'\b(?:MBG|BGN|Makanan\s*Bergizi\s*Gratis|Makan\s*Bergizi\s*Gratis|Makan\s*Gratis)\b', case=False, na=False)]


# Simpan hasil bersih
cleaned_file = "tempofix.csv"
df_filtered.to_csv(cleaned_file, index=False)

print(f"Pembersihan selesai! Data disimpan di {cleaned_file}")


Pembersihan selesai! Data disimpan di tempofix.csv


In [30]:
dt = pd.read_csv('tempofix.csv')
dt_subset = dt.iloc[0:126]  # Mengambil indeks 4 sampai 157 (karena slicing di Python eksklusif di batas atas)
dt_subset.to_csv('tempofix1.csv', index=False)

In [ ]:
dt = pd.read_csv('tempofix1.csv')
dt = dt.dropna()

dt.to_csv('tempofixbgt.csv', index=False)

In [31]:
df = pd.read_csv('tempofix1.csv')
def extract_after_separator(text):
    if pd.isna(text):
        return text
    return re.split(r'\s*[-–—]+\s*', text, maxsplit=1)[-1]  # Pisah dengan semua variasi tanda "-" dan spasi

df["content"] = df["content"].apply(extract_after_separator)
df.to_csv('tempofixbgt.csv', index=False)

In [ ]:
# UNtuk ceeekk tempo

import os
import csv
import requests as req
from bs4 import BeautifulSoup as bs
from fake_useragent import UserAgent

ua = UserAgent()
hades = {'User-Agent': ua.random}

# Cek apakah User-Agent mengandung 'Windows'
while 'Windows' not in hades['User-Agent']:
    hades = {'User-Agent': ua.random}
url = f'https://www.tempo.co/search?q=%23mbg&page=2'
ge = req.get(url, hades).text
sop = bs(ge, 'html.parser')
li = sop.find('div', class_='flex flex-col divide-y divide-neutral-500')
print(li)
#lin = li.find_all('div')

#print(lin)


None


In [ ]:
#UNTUK CEK
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
import csv
import os
import requests as req
from bs4 import BeautifulSoup as bs
import csv
from fake_useragent import UserAgent
import os
import re

chrome_options = Options()
chrome_options.add_argument("--headless")  # Jalankan di latar belakang (opsional)
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

service = Service(ChromeDriverManager().install())  # ✅ WebDriver otomatis didownload
driver = webdriver.Chrome(service=service)

# Akses halaman yang ingin diambil data
driver.get("https://www.tempo.co/search?q=%23mbg&page=2")
time.sleep(3)
ge = driver.page_source
sop = bs(ge, 'html.parser')
li = sop.find('div', class_='flex flex-col divide-y divide-neutral-500')
lin = li.find_all('div')
for x in lin:
            base_url = "https://www.tempo.co"
            link = base_url + x.find('a')['href']
            jdl = x.find("p", class_="text-neutral-1200 text-base line-clamp-4")
            headline = jdl.find('a', class_='hover:opacity-75').text.strip()
            #date_div = x.find('div', class_='news-date').text.strip()  # Contoh: "Sabtu, 15 Feb 2025 12:30 WIB"
            #date = 77
            #print(link)
            ge_ = req.get(link, hades).text
            sop_ = bs(ge_, 'html.parser')
            content = sop_.find_all('div', class_='max-lg:container xl')
            #print(content)
            date_div = sop_.find('p', class_='text-neutral-900 text-sm').text.strip()
            match = re.findall(r"([\d]{1,2} \w+ \d{4})", date_div)
            #print(match)

            for x in content:
                    x = x.find_all('p')
                    y  = [y.text for y in x]
                    content_ = ' '.join(y).replace('\n', '').replace('ADVERTISEMENT', '').replace('SCROLL TO CONTINUE WITH CONTENT', '')
                    print(content_)


            


In [10]:
print(headline)

In [14]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Inisialisasi driver
driver = webdriver.Chrome()  # Pastikan chromedriver sudah terinstal
driver.get("https://www.cnnindonesia.com/search/?query=makanan%20bergizi%20gratis")  # Ganti dengan URL situs yang ingin di-scrape

# Tunggu sampai elemen artikel yang asli muncul (bukan placeholder)
try:
    WebDriverWait(driver, 100).until(
        EC.presence_of_element_located((By.CLASS_NAME, "flex-grow"))
    )
    
    # Ambil semua artikel yang sudah termuat
    articles = driver.find_elements(By.CLASS_NAME, "flex-grow")

    for article in articles:
        try:
            # Cari elemen judul
            title = article.find_element(By.TAG_NAME, "h2").text
            link = article.find_element(By.TAG_NAME, "a").get_attribute("href")

            print(f"Judul: {title}")
            print(f"Link: {link}\n")
        except:
            print("Gagal mengambil data artikel")
finally:
    driver.quit()  # Tutup browser setelah selesai


Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambil data artikel
Gagal mengambi

In [ ]:
print(content)

In [45]:
ge_ = req.get(link, hades).text
sop_ = bs(ge_, 'html.parser')
content = sop_.find_all('article', class_='detailsContent force-17 mt40')

for x in content:
    x = x.find_all('p')
    y  = [y.text for y in x]
    content_ = ' '.join(y).replace('\n', '').replace('ADVERTISEMENT', '').replace('SCROLL TO CONTINUE WITH CONTENT', '')
    print(content_)
   

Bisnis.com, BANDUNG — Kota Bandung terpilih menjadi salah satu pilot project Program Makan Bergizi Gratis di Sekolah yang dilaksanakan oleh Indonesia Food Security Review (IFSR). Rencananya, IFSR menyiapkan 2.500 porsi makanan bergizi gratis bagi siswa di 6 sekolah di Kecamatan Arcamanik Kota Bandung selama 90 hari mulai Juli hingga September 2024 mendatang. Keenam sekolah yang terpilih menjadi lokus program tersebut yakni SDS Rancakasumba, Kelurahan Cisaranten Kulon, SDN 004 Cisaranten Kulon, Kelurahan Cisaranten Kulon, SDN 244 Guruminda, Kelurahan Cisaranten Kulon, SDN 180 Prakasa Nugraha, Kelurahan Cisaranten Bina Harapan, SMP Langlangbuana 2, Kelurahan Sukamiskin dan SDN 181 Sukamiskin, Kelurahan Sukamiskin. Plh Sekretaris Daerah Kota Bandung Hikmat Ginanjar mengatakan Pemkot Bandung akan mendukung program Makan Bergizi Gratis di sekolah. Hal tersebut diungkapkan Hikmat saat menerima Audiensi dari Indonesia Food Security Review terkait Uji Coba Program Makan Bergizi Gratis di Sekol

In [70]:
link = f'https://banten.idntimes.com/news/banten/m-iqbal-1/dapur-mbg-di-tangsel-terbakar-kerugian-ditaksir-capai-rp100-juta'
ge_1 = req.get(link, hades).text
sop_1 = bs(ge_1, 'html.parser')
masuklagi = sop_1.find('div', class_='read-more-btn-check').find('a')['href']

ge_ = req.get(masuklagi, hades).text
sop_ = bs(ge_, 'html.parser')
content = sop_.find_all('div', class_='article-description')

content_=""

for x in content:
        x = x.find_all('p')
        y  = [y.text for y in x]
        content_ = ' '.join(y).replace('\n', '').replace('ADVERTISEMENT', '').replace('SCROLL TO CONTINUE WITH CONTENT', '')

print(content_)

Tangerang Selatan, IDN Times - Dapur makan bergizi gratis (MBG) di Kecamatan Pamulang, Tangerang Selatan (Tangsel), terbakar Selasa (18/2/2025) dini hari. Akibat kejadian ini, sebagian sumber bahan pangan untuk program tersebut musnah. Komandan Peleton (Danton) Dinas Pemadam Kebakaran Tangsel, Imam, mengungkapkan peristiwa kebakaran di dapur MBG Pamulang terjadi sekitar pukul 02.43 WIB. “Benar kebakaran di Dapur Makan Bergizi Gratis. Kejadiannya jam 2 pagi,” kata Imam dikonfirmasi, Selasa (18/2/2025).    Baca Juga: Kader Gerindra Tangsel Diminta Kawal Program Kerja Prabowo    Imam menyebutkan, dalam proses pemadaman dapur MBG di Pamulang itu, 9 personel Damkar Tangsel diterjunkan hasilnya api dapat dipadamkan satu jam setelahnya. “Pukul 02.55 WIB mulai padamin. Selesai pemadaman pukul 03.45 WIB,” kata dia. Imam juga memastikan tidak ada korban jiwa dalam peristiwa tersebut, namun untuk kerugian mencapai Rp100 juta. “Korban nihil. Kerugian Rp100 juta,” ucapnya. Selain bahan makanan musn

In [71]:
print(content)

[<div class="article-description has-listicle has-keypoint" id="article-description">
<div class="mt-10">
<div class="follow-hyperlink">
    Follow IDN Times untuk mendapatkan informasi terkini. Klik untuk follow 
    <a href="http://bit.ly/wachannel-idntimes" rel="nofollow noreferrer" target="_blank">WhatsApp Channel</a> &amp; 
    <a href="https://bit.ly/googlenews-idntimes" rel="nofollow noreferrer" target="_blank">Google News</a>
</div>
</div>
<div class="main-description has-keypoint"><p><strong><a href="https://www.idntimes.com/tag/tangerang-selatan" rel="dofollow" target="_blank">Tangerang Selatan</a>, IDN Times</strong> - Dapur <a href="https://www.idntimes.com/tag/makan-bergizi-gratis" rel="dofollow" target="_blank">makan bergizi gratis</a> (MBG) di Kecamatan Pamulang, Tangerang Selatan (<a href="https://www.idntimes.com/tag/tangsel" rel="dofollow" target="_blank">Tangsel</a>), terbakar Selasa (18/2/2025) dini hari. Akibat kejadian ini, sebagian sumber bahan pangan untuk progr

In [29]:
#INI  DETIK
import os
import csv
import requests as req
from bs4 import BeautifulSoup as bs
from fake_useragent import UserAgent

ua = UserAgent()
hades = {'User-Agent': ua.random}

# Cek apakah User-Agent mengandung 'Windows'
while 'Windows' not in hades['User-Agent']:
    hades = {'User-Agent': ua.random}


def scrape_detik(hal):
    global hades
    a = 0
    file_path = 'detik.csv'
    
    # Cek apakah file sudah ada
    file_exists = os.path.isfile(file_path)

    for page in range(1, hal + 1):
        url = f'https://www.detik.com/search/searchnews?query=makanan%20bergizi%20gratis&page={page}&result_type=relevansi&fromdatex=07/01/2025&todatex=28/02/2025'
        ge = req.get(url, hades).text
        sop = bs(ge, 'html.parser')
        li = sop.find('div', class_='list-content')
        lin = li.find_all('article')

        with open(file_path, 'a', newline='', encoding='utf-8') as file:
            wr = csv.writer(file, delimiter=',')
            
            # Jika file belum ada, tulis header terlebih dahulu
            if not file_exists:
                wr.writerow(["headline", "date", "link", "content"])
                file_exists = True  # Hindari penulisan header ulang

            for x in lin:
                link = x.find('a')['href']
                date_div = x.find('div', class_='media__date').find('span').get('title')  # Contoh: "Sabtu, 15 Feb 2025 12:30 WIB"
                date = " ".join(date_div.split()[1:4])
                headline = x.find('a')['dtr-ttl']
                
                ge_ = req.get(link, hades).text
                sop_ = bs(ge_, 'html.parser')
                content = sop_.find_all('div', class_='detail__body-text itp_bodycontent')

                for x in content:
                    x = x.find_all('p')
                    y  = [y.text for y in x]
                    content_ = ' '.join(y).replace('\n', '').replace('ADVERTISEMENT', '').replace('SCROLL TO CONTINUE WITH CONTENT', '')

                    print(f'done[{a}] > {headline[:10]}')
                    a += 1
                    wr.writerow([headline, date, link, content_])

import lxml
scrape_detik(138)


done[0] > 12 Siswa S
done[1] > Polres Pro
done[2] > Viral Ada 
done[3] > Siswa Kota
done[4] > Anak Sekol
done[5] > Kapolres P


KeyboardInterrupt: 

In [31]:
# INI DETIK BARU
import os
import csv
import requests as req
from bs4 import BeautifulSoup as bs
from fake_useragent import UserAgent

ua = UserAgent()
hades = {'User-Agent': ua.random}

# Cek apakah User-Agent mengandung 'Windows'
while 'Windows' not in hades['User-Agent']:
    hades = {'User-Agent': ua.random}

def scrape_detik(hal):
    global hades
    a = 0
    file_path = 'detik.csv'

    # Cek apakah file sudah ada
    file_exists = os.path.isfile(file_path)

    for page in range(1, hal + 1):
        url = f'https://www.detik.com/search/searchnews?query=makanan%20bergizi%20gratis&page={page}&result_type=relevansi&fromdatex=07/01/2025&todatex=28/02/2025'
        ge = req.get(url, hades).text
        sop = bs(ge, 'html.parser')
        li = sop.find('div', class_='list-content')
        lin = li.find_all('article')

        with open(file_path, 'a', newline='', encoding='utf-8') as file:
            wr = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)

            # Jika file belum ada, tulis header terlebih dahulu
            if not file_exists:
                wr.writerow(["headline", "date", "link", "content"])
                file_exists = True  # Hindari penulisan header ulang

            for x in lin:
                link = x.find('a')['href']
                date_div = x.find('div', class_='media__date').find('span').get('title')  # Format: "Sabtu, 15 Feb 2025 12:30 WIB"
                date = " ".join(date_div.split()[1:4])
                headline = x.find('a').get('dtr-ttl', 'No Title')  # Gunakan get untuk menghindari error jika atribut tidak ada

                ge_ = req.get(link, hades).text
                sop_ = bs(ge_, 'html.parser')
                content = sop_.find_all('div', class_='detail__body-text itp_bodycontent')

                for x in content:
                    x = x.find_all('p')
                    y = [y.text.strip() for y in x]
                    content_ = ' '.join(y).replace('ADVERTISEMENT', '').replace('SCROLL TO CONTINUE WITH CONTENT', '')

                    print(f'done[{a}] > {headline[:10]}')
                    a += 1
                    wr.writerow([headline, date, link, content_])

import lxml
scrape_detik(138)


done[0] > 12 Siswa S
done[1] > Polres Pro
done[2] > Viral Ada 
done[3] > Siswa Kota
done[4] > Anak Sekol
done[5] > Kapolres P
done[6] > Bocoran Me
done[7] > Program Ma
done[8] > Saat Bumil
done[9] > Makanan Be
done[10] > Koperasi S
done[11] > Gibran Cek
done[12] > Peringati 
done[13] > 3.028 Sisw
done[14] > Dukung Ast
done[15] > Kala Kepal
done[16] > Anggaran M
done[17] > Makan Berg
done[18] > Pemerintah
done[19] > Video: Kat
done[20] > Sampel Men
done[21] > Sorotan Do
done[22] > Menu MBG d
done[23] > Ada Siswa 
done[24] > BUMN Ikut 
done[25] > Tolak Maka
done[26] > Cak Imin H
done[27] > Kepala BGN
done[28] > Anggaran M
done[29] > Video: Men
done[30] > Makan Berg
done[31] > Makan Berg
done[32] > APBN Cukup
done[33] > Bocoran Re
done[34] > Polri, BGN
done[35] > Kepala BGN
done[36] > Program MB
done[37] > Kumpulan M
done[38] > Pangan Lok
done[39] > Ratusan Ma
done[40] > Anggaran M
done[41] > Makanan Be
done[42] > Kesibukan 
done[43] > Terdampak 
done[44] > Haidar Alw
done[45] > Konsep MB

In [33]:
import pandas as pd

dt = pd.read_csv('detik.csv')
dt = dt.dropna()
dt.head(10)



,headline,date,link,content
0,12 Siswa SD di Takalar Diduga Keracunan Makana...,27 Feb 2025,https://www.detik.com/sulsel/berita/d-7798432/...,Sebanyak 12 siswa SD di Kecamatan Mangarabomba...
1,Polres Probolinggo Gandeng Mahasiswa Berbagi M...,25 Feb 2025,https://www.detik.com/jatim/berita/d-7795192/p...,Dalam rangka mendukung progam Asta Cita Presid...
2,Viral Ada Menu Makan Bergizi Gratis Belum Mata...,28 Feb 2025,https://news.detik.com/berita/d-7800792/viral-...,Kepala Badan Gizi Nasional (BGN) Dadan Hindaya...
3,Siswa Kota Bima Dua Hari Tak Terima Makan Berg...,28 Feb 2025,https://www.detik.com/bali/nusra/d-7800541/sis...,Program Makan Bergizi Gratis (MBG) di Kota Bim...
4,Anak Sekolah Tetap Dapat Makan Bergizi Gratis ...,28 Feb 2025,https://www.detik.com/edu/sekolah/d-7799506/an...,Kepala Badan Gizi Nasional (BGN) Dadan Hidayan...
5,Kapolres Probolinggo Bagikan Makanan Bergizi G...,10 Jan 2025,https://www.detik.com/jatim/berita/d-7727047/k...,Kapolres Probolinggo AKBP Wisnu Wardana dan ja...
6,"Bocoran Menu MBG Saat Puasa dari Kepala BGN, D...",25 Feb 2025,https://www.detik.com/edu/sekolah/d-7794847/bo...,"Kepala Badan Gizi Nasional (BGN), Dadan Hinday..."
7,Program Makan Bergizi Gratis di Palembang Teta...,27 Feb 2025,https://www.detik.com/sumbagsel/berita/d-77971...,Program makan bergizi gratis (MBG) di Kota Pal...
8,Saat Bumil-Lansia di Ciparay Bandung Mendapat ...,18 Feb 2025,https://www.detik.com/jabar/berita/d-7783738/s...,"Para ibu hamil, ibu menyusui, balita, dan lans..."
9,"Makanan Bergizi Gratis, Jembatan Menuju RI Beb...",15 Jan 2025,https://health.detik.com/berita-detikhealth/d-...,Program Makan Bergizi Gratis (MBG) menjadi uju...


In [35]:
# Filter headline yang mengandung "MBG" atau "Makanan Bergizi Gratis"
dt_filtered = dt[dt['headline'].str.contains(r'\b(?:MBG|BGN|Makanan\s*Bergizi\s*Gratis|Makan\s*Bergizi\s*Gratis|Makan\s*Gratis)\b', case=False, na=False)]
dt_filtered = pd.DataFrame(dt_filtered)
dt_filtered.to_csv('detikfixbgt.csv', index=False, encoding='utf-8')

In [8]:
#INI IDN
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
import csv
import os
import requests as req
from bs4 import BeautifulSoup as bs
import csv
from fake_useragent import UserAgent
import os

chrome_options = Options()
chrome_options.add_argument("--headless")  # Jalankan di latar belakang (opsional)
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

service = Service(ChromeDriverManager().install())  # ✅ WebDriver otomatis didownload
driver = webdriver.Chrome(service=service)

# Akses halaman yang ingin diambil data
driver.get("https://www.idntimes.com/tag/makan-bergizi-gratis")

# Memberi waktu agar halaman memuat dengan baik
time.sleep(3)

# Melakukan scroll ke bawah
body = driver.find_element(By.TAG_NAME, 'body')

# Tentukan berapa kali scroll yang diinginkan (misalnya scroll 5 kali)
for _ in range(5):
    body.send_keys(Keys.PAGE_DOWN)  # Scroll ke bawah
    time.sleep(2)  # Tunggu agar konten baru dimuat

# Ambil HTML setelah scroll
html = driver.page_source

ua = UserAgent()
hades = {'User-Agent': ua.random}

# Cek apakah User-Agent mengandung 'Windows'
while 'Windows' not in hades['User-Agent']:
    hades = {'User-Agent': ua.random}


def scrape_detik():
    global hades
    a = 0
    file_path = 'idn1.csv'
    
    # Cek apakah file sudah ada
    file_exists = os.path.isfile(file_path)

    ge = html
    sop = bs(ge, 'html.parser')
    li = sop.find('div', class_='left-content')
    lin = li.find_all('div', class_='box-latest box-list')


    with open(file_path, 'a', newline='', encoding='utf-8') as file:
        wr = csv.writer(file, delimiter=',')
        
        # Jika file belum ada, tulis header terlebih dahulu
        if not file_exists:
            wr.writerow(["headline", "date", "link", "content"])
            file_exists = True  # Hindari penulisan header ulang

        for x in lin:
            link = x.find('a')['href']
            headline = x.find('a')['title']
            date_div = x.find('time', class_='date').text.strip()
            date = date_div
            
            ge_1 = req.get(link, hades).text
            sop_1 = bs(ge_1, 'html.parser')
            masuklagi = sop_1.find('div', class_='read-more-btn-check').find('a')['href']

            ge_ = req.get(masuklagi, hades).text
            sop_ = bs(ge_, 'html.parser')
            content = sop_.find_all('div', class_='article-description')

            for x in content:
                x = x.find_all('p')
                y  = [y.text for y in x]
                content_ = ' '.join(y).replace('\n', '').replace('ADVERTISEMENT', '').replace('SCROLL TO CONTINUE WITH CONTENT', '')

                print(f'done[{a}] > {headline[:10]}')
                a += 1
                wr.writerow([headline, date, link, content_])

scrape_detik()

done[0] > Respons Is
done[1] > Cek Banjir
done[2] > Siapa Peja
done[3] > Pemkot Par
done[4] > Pelajar di
done[5] > Temuan KPK
done[6] > Puasa, Sis
done[7] > Menu MBG d
done[8] > KPK Soroti
done[9] > DPR: Dapur


In [ ]:
# IDN BARUUU
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
import csv
import os
import requests as req
from bs4 import BeautifulSoup as bs
from fake_useragent import UserAgent

chrome_options = Options()
chrome_options.add_argument("--headless")  
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

service = Service(ChromeDriverManager().install())  
driver = webdriver.Chrome(service=service)

# Akses halaman IDN Times
driver.get("https://www.idntimes.com/tag/makan-bergizi-gratis")
time.sleep(3)

# Adaptif scrolling: scroll sampai semua artikel termuat
prev_height = driver.execute_script("return document.body.scrollHeight")
scroll_attempts = 0
max_attempts = 15  

while scroll_attempts < max_attempts:
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)  
    time.sleep(3)  
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    if new_height == prev_height:
        print("✅ Tidak ada konten baru yang dimuat, berhenti scrolling.")
        break  
    
    prev_height = new_height
    scroll_attempts += 1

print("✅ Selesai scrolling, mulai scraping data...")

# Ambil HTML setelah scroll selesai
html = driver.page_source
driver.quit()

ua = UserAgent()
hades = {'User-Agent': ua.random}

while 'Windows' not in hades['User-Agent']:
    hades = {'User-Agent': ua.random}

def scrape_idn():
    global hades
    a = 0
    file_path = 'idn2.csv'
    
    # Cek apakah file sudah ada
    file_exists = os.path.isfile(file_path)

    sop = bs(html, 'html.parser')
    li = sop.find('div', class_='left-content')
    lin = li.find_all('div', class_='box-latest box-list')

    with open(file_path, 'a', newline='', encoding='utf-8') as file:
        wr = csv.writer(file, delimiter=',')
        
        if not file_exists:
            wr.writerow(["headline", "date", "link", "content"])
            file_exists = True  

        for x in lin:
            link = x.find('a')['href']
            headline = x.find('a')['title']
            date_div = x.find('time', class_='date').text.strip()
            date = date_div
            
            ge_1 = req.get(link, hades).text
            sop_1 = bs(ge_1, 'html.parser')
            masuklagi = sop_1.find('div', class_='read-more-btn-check').find('a')['href']

            ge_ = req.get(masuklagi, hades).text
            sop_ = bs(ge_, 'html.parser')
            content = sop_.find_all('div', class_='article-description')

            for x in content:
                x = x.find_all('p')
                y  = [y.text for y in x]
                content_ = ' '.join(y).replace('\n', '').replace('ADVERTISEMENT', '').replace('SCROLL TO CONTINUE WITH CONTENT', '')

                print(f'done[{a}] > {headline[:10]}')
                a += 1
                wr.writerow([headline, date, link, content_])

scrape_idn()


✅ Tidak ada konten baru yang dimuat, berhenti scrolling.
✅ Selesai scrolling, mulai scraping data...
done[0] > Respons Is
done[1] > Cek Banjir
done[2] > Siapa Peja
done[3] > Pemkot Par
done[4] > Pelajar di
done[5] > Temuan KPK
done[6] > Puasa, Sis
done[7] > Menu MBG d
done[8] > KPK Soroti
done[9] > DPR: Dapur
done[10] > Sosialisas
done[11] > RI Minat E
done[12] > Menu Progr
done[13] > Siswa samp
done[14] > MBG di Pal
done[15] > Badan Gizi
done[16] > MBG di Sur
done[17] > Prabowo Re
done[18] > Siswa Pale
done[19] > Kapan MBG 
done[20] > Program Ma
done[21] > Kepala BGN
done[22] > Siswa di B
done[23] > Evaluasi M
done[24] > 4 Menu Mak
done[25] > Suara Sisw
done[26] > Program Ma
done[27] > Tantangan 
done[28] > Gibran Sin
done[29] > 12 Siswa d
done[30] > Pelaksanaa
done[31] > Ombudsman 
done[32] > Ibu Hamil 
done[33] > Menteri BK
done[34] > Program MB
done[35] > Menteri Wi
done[36] > Budi Arie 
done[37] > SPPG Gagak
done[38] > Budi Arie 
done[39] > Menu Makan
done[40] > MBG Bulan 
done[41

In [24]:
dt = pd.read_csv('idn2.csv')
dtuy = dt.iloc[21:194]
def extract_after_separator(text):
    if pd.isna(text):
        return text
    return re.split(r'\s*[-–—]+\s*', text, maxsplit=1)[-1]  # Pisah dengan semua variasi tanda "-" dan spasi

dtuy["content"] = dtuy["content"].apply(extract_after_separator)
dtuy.to_csv('idngab1.csv', index=False, encoding='utf-8')

C:\Users\raziq\AppData\Local\Temp\ipykernel_27060\2990037096.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtuy["content"] = dtuy["content"].apply(extract_after_separator)


In [21]:
dt1 = pd.read_csv('idn.csv')
dtkrop = dt1.iloc[132:203]
def extract_after_separator(text):
    if pd.isna(text):
        return text
    return re.split(r'\s*[-–—]+\s*', text, maxsplit=1)[-1]  # Pisah dengan semua variasi tanda "-" dan spasi
dtkrop['content']=dtkrop['content'].apply(extract_after_separator)
dtkrop.to_csv('idngab.csv', index=False, encoding='utf-8')

C:\Users\raziq\AppData\Local\Temp\ipykernel_27060\3674090524.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dtkrop['content']=dtkrop['content'].apply(extract_after_separator)


In [25]:
df1 = pd.read_csv('idngab1.csv')
df2 = pd.read_csv('idngab.csv')
df_combined = pd.concat([df1, df2], ignore_index=True)
df_combined.to_csv('idnfixbgt.csv', index=False, encoding='utf-8')

In [40]:
#INI BISNIS
import os
import csv
import requests as req
from bs4 import BeautifulSoup as bs
from fake_useragent import UserAgent

ua = UserAgent()
hades = {'User-Agent': ua.random}

# Cek apakah User-Agent mengandung 'Windows'
while 'Windows' not in hades['User-Agent']:
    hades = {'User-Agent': ua.random}


def scrape_bisnis(hal):
    global hades
    a = 0
    file_path = 'bisnis1.csv'
    
    # Cek apakah file sudah ada
    file_exists = os.path.isfile(file_path)

    for page in range(1, hal + 1):
        url = f'https://search.bisnis.com/?q=makan%20bergizi%20gratis&page={page}'
        ge = req.get(url, hades).text
        sop = bs(ge, 'html.parser')
        li = sop.find('div', class_='artWrap -row -center')
        lin = li.find_all('div', class_ = 'artItem')

        with open(file_path, 'a', newline='', encoding='utf-8') as file:
            wr = csv.writer(file, delimiter=',')
            
            # Jika file belum ada, tulis header terlebih dahulu
            if not file_exists:
                wr.writerow(["headline", "date", "link", "content"])
                file_exists = True  # Hindari penulisan header ulang

            for x in lin:
                link = x.find('a')['href']
                headline = x.find('h4', class_='artTitle').text.strip()
                date_div = x.find('div', class_='artDate').text.strip()  # Contoh: "Sabtu, 15 Feb 2025 12:30 WIB"
                date = date_div
                
                
                ge_ = req.get(link, hades).text
                sop_ = bs(ge_, 'html.parser')
                content = sop_.find_all('article', class_='detailsContent force-17 mt40')


                for x in content:
                    x = x.find_all('p')
                    y  = [y.text for y in x]
                    content_ = ' '.join(y).replace('\n', '').replace('ADVERTISEMENT', '').replace('SCROLL TO CONTINUE WITH CONTENT', '')

                    print(f'done[{a}] > {headline[:10]}')
                    a += 1
                    wr.writerow([headline, date, link, content_])

import lxml
scrape_bisnis(10)


done[0] > KPK Siap K
done[1] > Layanan Gu
done[2] > Gubernur M
done[3] > Pemerintah
done[4] > Makan Berg
done[5] > Kepala BGN
done[6] > Siswa Kera
done[7] > Dicanangka
done[8] > Hashim: Su
done[9] > RI Buka Pe
done[10] > 167 Perusa
done[11] > Kepala BGN
done[12] > Program Ma
done[13] > Kepala BGN
done[14] > Luhut Sebu
done[15] > Mensesneg 
done[16] > PDIP: Sisw
done[17] > Mahasiswa 
done[18] > Anggaran M
done[19] > Ketahanan 
done[20] > Penuhi Pro
done[21] > Prabowo Al
done[22] > Dana Makan
done[23] > Prabowo Mi
done[24] > Prabowo Pa
done[25] > Makan Berg
done[26] > 37 Dapur U
done[27] > BGN Tegask
done[28] > Australia 
done[29] > Pemerintah
done[30] > Prabowo Ke
done[31] > Gibran Cur
done[32] > Banyak Sek
done[33] > OPM Ancam 
done[34] > Respons Ba
done[35] > Penerima M
done[36] > PBNU Bentu
done[37] > Prabowo Si
done[38] > Presiden P
done[39] > Hashim Kla
done[40] > Tambal Def
done[41] > Rencana Ma
done[42] > TNI Gelar 
done[43] > Anggaran B
done[44] > Ekonom Rag
done[45] > Sri Mulya

In [48]:
df = pd.read_csv('bisnis1.csv')
#df["content"] = df["content"].apply(lambda x: x.split(" - ", 1)[1] if " - " in x else x)
def extract_after_separator(text):
    separators = [" -", " --", " –", " ---", " —— "]
    for sep in separators:
        if sep in text:
            return text.split(sep, 1)[1].strip()  # Ambil teks setelah pemisah pertama
    return text
df["content"] = df["content"].apply(extract_after_separator)
df.to_csv('bisnisbersih.csv', index=False)

In [3]:
import re

df = pd.read_csv('bisnis1.csv')
def extract_after_separator(text):
    if pd.isna(text):
        return text
    return re.split(r'\s*[-–—]+\s*', text, maxsplit=1)[-1]  # Pisah dengan semua variasi tanda "-" dan spasi

df["content"] = df["content"].apply(extract_after_separator)


In [5]:
df.to_csv('bisnisbersiiih.csv', index=False, encoding='utf-8')

In [7]:
dt_subset = df.iloc[5:170]
dt_subset.to_csv('bisnisfixbgt.csv', index=False, encoding='utf-8')

In [43]:
#ini CNN

import os
import csv
import requests as req
from bs4 import BeautifulSoup as bs
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time
import csv
import os
import requests as req
from bs4 import BeautifulSoup as bs
import csv
from fake_useragent import UserAgent
import os

chrome_options = Options()
chrome_options.add_argument("--headless")  # Jalankan di latar belakang (opsional)
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

service = Service(ChromeDriverManager().install())  # ✅ WebDriver otomatis didownload
driver = webdriver.Chrome(service=service)

ua = UserAgent()
hades = {'User-Agent': ua.random}

# Cek apakah User-Agent mengandung 'Windows'
while 'Windows' not in hades['User-Agent']:
    hades = {'User-Agent': ua.random}


def scrape_detik(hal):
    global hades
    a = 0
    file_path = 'CNN1.csv'
    
    # Cek apakah file sudah ada
    file_exists = os.path.isfile(file_path)

    for page in range(20, hal + 1):
        url = f'https://www.cnnindonesia.com/search/?query=Makan%20Bergizi%20Gratis&kanal=nasional&page={page}'
        driver.get(url)
        time.sleep(3)
        ge = driver.page_source
        sop = bs(ge, 'html.parser')
        li = sop.find('div', class_='flex flex-col gap-5')
        lin = li.find_all('article')

        with open(file_path, 'a', newline='', encoding='utf-8') as file:
            wr = csv.writer(file, delimiter=',')
            
            # Jika file belum ada, tulis header terlebih dahulu
            if not file_exists:
                wr.writerow(["headline", "date", "link", "content"])
                file_exists = True  # Hindari penulisan header ulang

            for x in lin:
                link = x.find('a')['href']
                headline = x.find('h2', class_='text-cnn_black_light').text.strip()
                #print(link)
                
                driver.get(link)
                time.sleep(3)
                ge_ = driver.page_source
                sop_ = bs(ge_, 'html.parser')
                content = sop_.find('div', class_='detail-text text-cnn_black text-sm grow min-w-0')
                #print(content)
                date_div = sop_.find('div', class_='text-cnn_grey text-sm mb-4').text.strip()  # Contoh: "Sabtu, 15 Feb 2025 12:30 WIB"
                date = date_div
                x = content.find_all('p')
                y  = [y.text for y in x]
                content_ = ' '.join(y).replace('\n', '').replace('ADVERTISEMENT', '').replace('SCROLL TO CONTINUE WITH CONTENT', '')

                print(f'done[{a}] > {headline[:10]}')
                a += 1
                wr.writerow([headline, date, link, content_])
                

import lxml
scrape_detik(42)


done[0] > Ketum PBNU
done[1] > Makan Berg
done[2] > Pj Gubernu
done[3] > Menko Prat
done[4] > Hari Perta
done[5] > Celetuk Si


AttributeError: 'NoneType' object has no attribute 'text'

In [ ]:
# INI CNN BARU
import os
import csv
import requests as req
from bs4 import BeautifulSoup as bs
from fake_useragent import UserAgent
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time

chrome_options = Options()
chrome_options.add_argument("--headless")  # Jalankan di latar belakang (opsional)
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

service = Service(ChromeDriverManager().install())  # ✅ WebDriver otomatis didownload
driver = webdriver.Chrome(service=service)

ua = UserAgent()
hades = {'User-Agent': ua.random}

# Cek apakah User-Agent mengandung 'Windows'
while 'Windows' not in hades['User-Agent']:
    hades = {'User-Agent': ua.random}


def scrape_detik(hal):
    global hades
    a = 0
    file_path = 'CNN1.csv'
    
    # Cek apakah file sudah ada
    file_exists = os.path.isfile(file_path)

    for page in range(20, hal + 1):
        url = f'https://www.cnnindonesia.com/search/?query=Makan%20Bergizi%20Gratis&kanal=nasional&page={page}'
        driver.get(url)
        time.sleep(3)
        ge = driver.page_source
        sop = bs(ge, 'html.parser')
        li = sop.find('div', class_='flex flex-col gap-5')

        # Jika `li` tidak ditemukan, skip halaman ini
        if li is None:
            print(f"❌ Tidak menemukan daftar artikel di halaman {page}, skip...")
            continue

        lin = li.find_all('article')

        with open(file_path, 'a', newline='', encoding='utf-8') as file:
            wr = csv.writer(file, delimiter=',')
            
            # Jika file belum ada, tulis header terlebih dahulu
            if not file_exists:
                wr.writerow(["headline", "date", "link", "content"])
                file_exists = True  # Hindari penulisan header ulang

            for x in lin:
                try:
                    link = x.find('a')['href']
                    headline = x.find('h2', class_='text-cnn_black_light').text.strip()
                    
                    driver.get(link)
                    time.sleep(3)
                    ge_ = driver.page_source
                    sop_ = bs(ge_, 'html.parser')

                    # Cek apakah `content` ada, kalau tidak, skip artikel ini
                    content = sop_.find('div', class_='detail-text text-cnn_black text-sm grow min-w-0')
                    if content is None:
                        print(f"❌ Konten tidak ditemukan di {link}, skip...")
                        continue

                    # Cek apakah `date_div` ada, kalau tidak, skip artikel ini
                    date_div = sop_.find('div', class_='text-cnn_grey text-sm mb-4')
                    if date_div is None:
                        print(f"❌ Tanggal tidak ditemukan di {link}, skip...")
                        continue

                    date = date_div.text.strip()
                    x = content.find_all('p')
                    y = [y.text for y in x]
                    content_ = ' '.join(y).replace('\n', '').replace('ADVERTISEMENT', '').replace('SCROLL TO CONTINUE WITH CONTENT', '')

                    print(f'✅ done[{a}] > {headline[:10]}')
                    a += 1
                    wr.writerow([headline, date, link, content_])

                except Exception as e:
                    print(f"⚠️ Error saat memproses {link}: {e}")
                    continue  # Jika ada error, lanjutkan ke artikel berikutnya

import lxml
scrape_detik(42)


✅ done[0] > Ketum PBNU
✅ done[1] > Makan Berg
✅ done[2] > Pj Gubernu
✅ done[3] > Menko Prat
✅ done[4] > Hari Perta
✅ done[5] > Celetuk Si
❌ Tanggal tidak ditemukan di https://www.cnnindonesia.com/nasional/20250112122830-22-1186282/foto-pagar-laut-tangerang-hingga-sidang-gugatan-pilkada-di-mk, skip...
✅ done[6] > Hasto Soal
✅ done[7] > Program Ma
✅ done[8] > 5 Evaluasi
✅ done[9] > Makan Grat
❌ Tanggal tidak ditemukan di https://www.cnnindonesia.com/nasional/20250110141811-22-1185801/foto-kick-off-program-mbg-untuk-balita-dan-ibu-hamil-menyusui, skip...
✅ done[10] > BPOM Temuk
✅ done[11] > Siswa di M
✅ done[12] > SD dan PAU
✅ done[13] > Orang Tua 
✅ done[14] > Belum Semu
✅ done[15] > Murid SD J
✅ done[16] > Evaluasi M
✅ done[17] > Fakta-fakt
✅ done[18] > Makan Grat
✅ done[19] > Porsi Lauk
✅ done[20] > Kabupaten 
✅ done[21] > Cerita Mak
✅ done[22] > Siswa di P
✅ done[23] > Tak Habisk
✅ done[24] > Sudah Sara
✅ done[25] > Dapur Belu
✅ done[26] > Pemprov DK
✅ done[27] > Distribusi
✅ done[28]

In [48]:
dt1 = pd.read_csv('CNN.csv')
dt2 = pd.read_csv('CNN1.csv')
dt2 = dt2.iloc[6:]

dtg = pd.concat([dt1,dt2], ignore_index=True)
dtg1 = dtg[dtg['headline'].str.contains(r'\b(?:MBG|BGN|Makanan\s*Bergizi\s*Gratis|Makan\s*Bergizi\s*Gratis|Makan\s*Gratis)\b', case=False, na=False)]
dt_filtered = pd.DataFrame(dtg1)
dt_filtered.to_csv('CNNFIX.csv', index=False, encoding='utf-8')


In [51]:
dt = pd.read_csv('CNNFIX.csv')
dt = dt.iloc[9:117]
dt.to_csv('CNNFIXBGT.csv', index=False, encoding='utf-8')